## Notbook A: How to preprocess input data for simulation

#### Prerequisites:
- Basic knowledge of Python 

#### Goals:
- guide on how to prepare data as the simulation expects it.

#### Content overview: 
- Loading anatomy segmentation
- Writing XML rawdata input

First off, we require to pre-process our data.
We require the following minimum input data for a simulation:
- an anatomy segmentation in RAI format (right-anterior-inferior)
- an MR rawdata file in ISMRMRD format describing the segmentation (contrast template)
- an MR rawdata file in ISMRMRD format describing the acquisition process (acquisition template)
- an XML descriptor assigning tissue parameters to the labels.

In [ ]:

# first we generate where we store our data
import os
from pathlib import Path 

fpath_out = Path(os.getenv("SIRF_INSTALL_PATH"))
fpath_out = fpath_out / "share/SIRF-3.1/Simulation/Input/"

fpath_out.mkdir(exist_ok=True,parents=True)

### Segmentation

In [ ]:
import nibabel as nib

root_path = '/media/sf_CCPPETMR/TestData/Input/xDynamicSimulation/pDynamicSimulation/'
fpath_segmentation_nii = root_path + 'Cube128/label_volume_rai.nii'

segmentation = nib.load(fpath_segmentation_nii)
print("The data shape is {}".format(segmentation.shape))

Let's have a look at the data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

f, axs = plt.subplots(1,3)
axs[0].imshow(segmentation.get_fdata()[:,:,64])
axs[0].set_ylabel("L-R")
axs[0].set_xlabel("P-A")
axs[0].set_xticks([])
axs[0].set_yticks([])

axs[1].imshow(segmentation.get_fdata()[:,64,:])
axs[1].set_ylabel("L-R")
axs[1].set_xlabel("S-I")
axs[1].set_xticks([])
axs[1].set_yticks([])

axs[2].imshow(segmentation.get_fdata()[64,:,:])
axs[2].set_ylabel("P-A")
axs[2].set_xlabel("S-I")
axs[2].set_xticks([])
axs[2].set_yticks([])

plt.show()

RAI means: the data are available in memory such with increasing XYZ index the voxels move from left to right, from posteior to anterior, and from superior to inferior.
Now this needs to be stored with orientation as well as resolution information.

We still need to store this geometry information into the segmentation Nifti.

In [ ]:
import numpy as np

resolution_mm_per_pixel = np.array([2,2,-2,1])
offset_mm =(-np.array(segmentation.shape)/2 + 0.5) * resolution_mm_per_pixel[0:3]

affine = np.diag(resolution_mm_per_pixel)
affine[:3,3] = offset_mm

#
img = nib.Nifti1Image(segmentation.get_fdata(), affine)
fname_segmentation = str(fpath_out) + "/segmentation.nii"
nib.save(img, fname_segmentation)

### Motion Fields
The motion fields must match the segmentation.

In [ ]:
import sirf.Reg as pReg

def read_motionfields(fpath_prefix):
	p = sorted( Path(fpath_prefix).glob('mvf*') )
	files = [x for x in p if x.is_file()]
	
	temp = []
	for f in files:
		print("Reading from {} ... ".format(f))
		img = pReg.NiftiImageData3DDisplacement(str(f))
		temp.append(img)

	data = np.array(temp, dtype=object)
	return data

fpath_resp_mvf = root_path + 'Cube128/mvf_resp/'
resp_mvfs = read_motionfields(fpath_resp_mvf)


In [ ]:

inhale_mvf = resp_mvfs[-1] 
inhale_arr = inhale_mvf.as_array()
inhale_arr = np.linalg.norm(inhale_arr, axis=4)
print(inhale_arr.shape)

f, axs = plt.subplots(1,3)
axs[0].imshow(inhale_arr[:,:,64])
axs[0].set_ylabel("L-R")
axs[0].set_xlabel("P-A")
axs[0].set_xticks([])
axs[0].set_yticks([])
axs[0].xaxis.label.set_color('white')
axs[0].yaxis.label.set_color('white')

axs[1].imshow(inhale_arr[:,64,:])
axs[1].set_ylabel("L-R")
axs[1].set_xlabel("S-I")
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[1].xaxis.label.set_color('white')
axs[1].yaxis.label.set_color('white')


axs[2].imshow(inhale_arr[64,:,:])
axs[2].set_ylabel("P-A")
axs[2].set_xlabel("S-I")
axs[2].set_xticks([])
axs[2].set_yticks([])
axs[2].xaxis.label.set_color('white')
axs[2].yaxis.label.set_color('white')

plt.show()

### Raw data

The simulation needs two MR raw-data objects to serve as templates:
- one to set up the segmentation in image space, the so-called contrast template
- one to dictate the simulated acquisition, the so-called acquisition template

Since the simulation is able to resample different containers their geometry needs to be defined first. 
This is done by using a pre-compiled executable usually used for the SIRF tests.


In [ ]:
fpath_SIRF_build = "/home/sirfuser/devel/buildVM/builds/SIRF/build/"
fpath_preprocess_exe = fpath_SIRF_build + "src/xGadgetron/cGadgetron/tests/MR_PROCESS_TESTDATA"

fname_input_contrast_template = root_path + 'Cube128/CV_nav_cart_128Cube_FLASH_T1.h5'
fname_contrast_template = str(fpath_out) + "/contrast_template.h5"
command_conttempl = "{} {} {}".format(fpath_preprocess_exe, fname_contrast_template, fname_contrast_template)


fname_input_acquisition_template = root_path + 'General/meas_MID33_rad_2d_gc_FID78808_ismrmrd.h5'
fname_acquisition_template = str(fpath_out) + "/acquisition_template.h5"
command_acqtemplate = "{} {} {}".format(fpath_preprocess_exe, fname_input_acquisition_template, fname_acquisition_template)


In [ ]:

print("Running {}".format(command_conttempl))
os.system(command_conttempl)

os.system(command_acqtemplate)
print("Running {}".format(command_acqtemplate))

### XML descriptor 

Here is an example of the XML descriptor. 

It contains
- the label | which voxels it describes in the segmentation
- the name | keeps track of what we actually mean by it 
- MR parameters | T1, T2, proton density and chemical shift
- (PET parameters | only relevant for PET simulations)

<img src="./simulation_xml_example.png" alt="drawing" width=600 />



<h4> Features and Caveats: </h4>

- the XML descriptor **needs to come in the form as displayed**.
- arbitarily many sections of TissueParameter objects can be added, it does not matter if there are parameters described that do not appear in your segmentation.
- **all labels appearing in your segmentations need to appear**, otherwise an error will occur informing you of your mistake. The above example would require a couple of more entries.
- the PET parameters are not taken into consideration for MR simulations, but they still need to be present, fill them with zeros.


In [ ]:

import sirf.Reg as pReg
import sirf.DynamicSimulation as pDS
import sirf.Gadgetron as pMR


fpath_input = fpath_out

fpath_out = Path(os.getenv("SIRF_INSTALL_PATH"))
fpath_out = fpath_out / "share/SIRF-3.1/Simulation/Output/"
fpath_out.mkdir(exist_ok=True,parents=True)

prefix_ground_truth = str(fpath_out) + "/simulation_geometry_acquisition_offset_parametermap"


In [ ]:

# load the templates into SIRF MR acquisition data container
contrast_template = pMR.AcquisitionData(fname_contrast_template)
acquisition_template = pMR.AcquisitionData(fname_acquisition_template)


In [ ]:

# load the labels into SIRF Nifti Container
segmentation = pReg.NiftiImageData3D(fname_segmentation)

# set up the simulation with the segmentation and corresponding XML filename.
fname_xml = str(fpath_input) + '/XCAT_TissueParameters_XML.xml'
mrsim = pDS.MRDynamicSimulation(segmentation, fname_xml)

mrsim.set_contrast_template_data(contrast_template)
mrsim.set_acquisition_template_data(acquisition_template)


In [ ]:
# now we set up an affine transformation to move the acquired slice around
offset_x_mm = 0
offset_y_mm = 0
offset_z_mm = -127.5 
rotation_angles_deg = [0,0,0]
translation = np.array([offset_x_mm, offset_y_mm, offset_z_mm])
euler_angles_deg = np.array(rotation_angles_deg)

offset_trafo = pReg.AffineTransformation(translation, euler_angles_deg)
mrsim.set_offset_trafo(offset_trafo)

filenames_parametermaps = mrsim.save_parametermap_ground_truth(prefix_ground_truth)

In [ ]:
t1_map = pReg.NiftiImageData(filenames_parametermaps[0])
t2_map = pReg.NiftiImageData(filenames_parametermaps[1])
rho_map = pReg.NiftiImageData(filenames_parametermaps[2])

f, axs = plt.subplots(1,3)
axs[0].imshow(t1_map.as_array())
axs[0].axis("off")
axs[0].set_title("T1")

axs[1].imshow(t2_map.as_array())
axs[1].axis("off")
axs[1].set_title("T2")

axs[2].imshow(rho_map.as_array())
axs[2].axis("off")
axs[2].set_title("Spin density")

plt.show()
